In [1]:
from tensorflow.keras.models import load_model

import pandas as pd
import numpy as np

In [2]:
MODEL_DIR = './models/'

In [74]:
name = input('Enter model name: ')

model = load_model(MODEL_DIR + name + '_keras')
print('Loaded model: ' + name)
model

Loaded model: seq_1k


In [102]:
# Predict matchup
off_id = 1629027
def_id = 1629636

# Get offense stats
data = pd.read_csv('./stats/2021-22_last5_off_stats.csv')
data = data.add_prefix('OFF_')
data.rename(columns={'OFF_SEASON_ID': 'SEASON_ID'}, inplace=True)
inputs_o = data[data['OFF_PLAYER_ID'] == off_id]

# Get defensive stats
def_data = pd.read_csv('./stats/2021-22_last5_def_stats.csv')
def_data = def_data.fillna(0)
def_data = def_data.add_prefix('DEF_')
def_data.rename(columns={'DEF_SEASON_ID': 'SEASON_ID', 'DEF_DEF_RATING': 'DEF_RATING', 
                     'DEF_DEF_BOXOUTS': 'DEF_BOXOUTS'}, inplace=True)
inputs_d = def_data[def_data['DEF_PLAYER_ID'] == def_id]
print('Offensive player: ' + def_data[def_data['DEF_PLAYER_ID'] == off_id].reset_index()['DEF_PLAYER_NAME'][0])
print('Devensive player: ' + def_data[def_data['DEF_PLAYER_ID'] == def_id].reset_index()['DEF_PLAYER_NAME'][0])

# Get both inputs together
inputs_o.drop(columns=['OFF_PLAYER_ID', 'OFF_PTS_PER_100'], inplace=True)
inputs_d.drop(columns=['DEF_PLAYER_ID', 'DEF_PLAYER_NAME', 'SEASON_ID'], inplace=True)
inputs_o.drop(columns=['OFF_POST_TOUCH_PTS', 'OFF_POST_TOUCH_FG_PCT', 'OFF_FGM', 'OFF_PTS', 'OFF_EFF_FG_PCT', 'OFF_OREB', 'OFF_PAINT_TOUCH_FGA', 'OFF_PAINT_TOUCH_FGM', 'OFF_PAINT_TOUCH_TOV', 'OFF_PAINT_TOUCH_PASSES', 'OFF_PAINT_TOUCH_PTS', 'OFF_DREB', 'OFF_FG3M', 'OFF_FTM'], inplace=True)
inputs_d.drop(columns=['DEF_D_FGM', 'DEF_OPP_PTS_PAINT', 'DEF_CONTESTED_SHOTS_2PT', 'DEF_DEFLECTIONS', 'DEF_OPP_PTS_FB', 'DEF_OPP_PTS_2ND_CHANCE', 'DEF_OPP_PTS_OFF_TOV', 'DEF_PCT_PLUSMINUS', 'DEF_G'], inplace=True)

inputs_o = inputs_o.reset_index(drop=True)
inputs_d = inputs_d.reset_index(drop=True)
X_predict = pd.concat([inputs_d, inputs_o], axis=1)

X_predict.sort_index(axis=1, inplace=True)

# Normalize inputs
X_seqtest = pd.read_csv('./stats/X.csv')
X_seqtest.drop(columns=['OFF_POST_TOUCH_PTS', 'OFF_POST_TOUCH_FG_PCT', 'OFF_PTS', 'OFF_PTS_PER_100'], inplace=True)
X_seqtest.drop(columns=['OFF_MIN', 'OFF_GP', 'DEF_MIN', 'DEF_GP'], inplace=True)
# X_seqtest = X_seqtest[X_seqtest['SEASON_ID'] == 22021]

X_predict = (X_predict - X_seqtest.mean()) / X_seqtest.std()

X_predict = X_predict.fillna(0)

X_predict = X_predict.sort_index(axis=1)

# X_predict.columns
y = model.predict(X_predict.to_numpy())

# Convert from per 100 to predicted per game
off_data = data[data['OFF_PLAYER_ID'] == off_id].reset_index()
poss = (off_data['OFF_PTS'] / (off_data['OFF_PTS_PER_100'] / 100))[0]

y[0][0] * poss / 100

Offensive player: Trae Young
Devensive player: Darius Garland


13.959314753708329

In [ ]:
tmin = 41.0 + 30.0 + 29.7 + 25.3

27.263672 * 41.0/tmin + 20.496445 * 30.0/tmin + 26.41703 * 29.7/tmin + 21.454145 * 25.3/tmin

24.286345726190476

In [81]:
off_id = 202691

data = pd.read_csv('./stats/2021-22_last5_off_stats.csv')
data = data[data['PLAYER_ID'] == off_id]
data['PTS']

229    23.6
Name: PTS, dtype: float64

In [46]:
test = pd.read_csv('./stats/y.csv')
test = test[test['OFF_PLAYER_ID'] == 202691]
test = test[test['SEASON_ID'] == 22021]
test

,SEASON_ID,OFF_PLAYER_ID,OFF_PLAYER_NAME,DEF_PLAYER_ID,DEF_PLAYER_NAME,GP,MATCHUP_MIN,PARTIAL_POSS,PLAYER_PTS,TEAM_PTS,...,HELP_FGM,HELP_FGA,HELP_FG_PERC,MATCHUP_FTM,MATCHUP_FTA,SFL,MATCHUP_TIME_SEC,PLAYER_PPG,OFF_PTS,OFF_PTS_PER_100
13,22021,202691,Klay Thompson,2544,LeBron James,2,2:33,16.9,17.751479,94.674556,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,902.366864,10.465856,18.1,30.7
220,22021,202691,Klay Thompson,2772,Trevor Ariza,1,2:49,13.6,22.058824,80.882353,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,1244.117647,13.005365,18.1,30.7
922,22021,202691,Klay Thompson,201145,Jeff Green,2,3:27,18.0,11.111111,83.333333,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,1149.444444,6.550851,18.1,30.7
1057,22021,202691,Klay Thompson,201566,Russell Westbrook,2,3:01,18.3,10.928962,60.109290,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,991.256831,6.443460,18.1,30.7
2132,22021,202691,Klay Thompson,201950,Jrue Holiday,1,3:15,15.7,57.324841,146.496815,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,1240.764331,33.797382,18.1,30.7
2376,22021,202691,Klay Thompson,201988,Patty Mills,1,5:18,26.2,26.717557,114.503817,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,1212.595420,15.752045,18.1,30.7
3622,22021,202691,Klay Thompson,202704,Reggie Jackson,2,4:09,23.0,8.695652,82.608696,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,1080.869565,5.126753,18.1,30.7
4365,22021,202691,Klay Thompson,203085,Austin Rivers,2,2:03,9.6,72.916667,239.583333,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,1280.208333,42.989957,18.1,30.7
4457,22021,202691,Klay Thompson,203090,Maurice Harkless,1,4:08,21.5,13.953488,111.627907,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,1153.488372,8.226649,18.1,30.7
4495,22021,202691,Klay Thompson,203095,Evan Fournier,1,4:12,26.7,18.726592,89.887640,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,943.820225,11.040759,18.1,30.7
